#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "eeS07GRd50jqPKjdtINo6jBfcPXS2tig"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive/松尾研/geniac/データチーム/インストラクションデータ"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備

多肢選択式でのプロンプト例

In [ ]:
PROMPT_TEMPLATE = """
以下のタスク指示において、タスク指示例のとおりに10種類のA,B,C,Dの4つの選択肢が含まれた質問文（input）と回答文（output）を生成してください。
回答は、回答フォーマット例のとおり、"id", "instruction", "input", "output"をJson形式で出力してください。

要件は以下の通りです：
1. 各文で動詞を繰り返さないようにしてください。
2. 質問文は、多様なビジネスシーンを想定してください。
3. ビジネスで使用する言葉遣いで回答してください。
4. です・ます調で回答してください。
5. 全ての文は100語以内にしてください。
6. 回答はA,B,C,Dのいずれかで回答してください。
7. "id"は1以上の整数にしてください。
8. "instruction"は回答フォーマット例のとおりすべて固定にしてください。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON)：
[
  {
    "id": 1,
    "instruction": "以下の質問文（input）に対して回答（output）してください。
質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。",
    "input": "<質問文>",
    "output": "<A or B or C or D>"
  },
  {
    "id": 2,
    "instruction": "以下の質問文（input）に対して回答（output）してください。
質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。",
    "input": "<質問文>",
    "output": "<A or B or C or D>"
  },
  {
    "id": 3,
    "instruction": "以下の質問文（input）に対して回答（output）してください。
質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。",
    "input": "<質問文>",
    "output": "<A or B or C or D>"
  }
]
"""

In [ ]:
FEWSHOT_EXAMPLES = [
  {
    "id": 1,
    "instruction": """以下の質問文（input）に対して回答（output）してください。
質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。""",
    "input": """商品の送付先を間違ったときの謝罪メールの冒頭にふさわしい文はどれですか。
A) このたびは、ABC電気でお新しいPCをご購入いただき、誠にありがとうございます。お選びいただいたPCは、最新技術を駆使しており、ビジネス活動から日常の使用まで、幅広いニーズにお応えする製品です。
B) いつも大変お世話になっております。ABC社の陳です。この度は、私どもの管理不足により、お貸しいただいたPCに不慮の損害を与えてしまい、誠に申し訳ございませんでした。
C) このたびは、ご注文いただいた商品と異なる商品をお届けしてしまい、大変なご迷惑をおかけしたことを心からお詫び申し上げます。ご注文いただいたXXXではなく、誤ってYYYを送付してしまいました。
D) いつも弊社をご利用いただき、誠にありがとうございます。この度は、お手元にお届けすべき商品を誤った住所に送付してしまい、大変なご迷惑をおかけしたことを心からお詫び申し上げます。""",
    "output": "D"
  },
  {
    "id": 2,
    "instruction": """以下の質問文（input）に対して回答（output）してください。
質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。""",
    "input": """市場調査結果に基づいて、会社の新規事業展開に関する意見を述べている文章です。この意見を主張している人物の立場は次のうちどれですか。
A) 会長
B) 営業部門部長
C) 経理部門部長
D) 企画部門係長""",
    "output": "D"
  },
  {
    "id": 3,
    "instruction": """以下の質問文（input）に対して回答（output）してください。
質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。""",
    "input": """会議における発言例を選択してください。前日、サービスの不具合があったことが分かったと仮定して、次のうち、最も適切な発言を選択してください。
A) それでは、今回の件についての報告をさせていただきます。
B) 私たちのサービスは常に信頼できるものであると言い切ってはいけません。
C) 不具合を確認させていただきました。これからの対応方針についてご覧ください。
D) 私たちは素晴らしいプロダクトを提供していると自惚れにもなりやすいことですが、しかしながら今回のような事故は発生してはいけません。""",
    "output": "C"
  },
  {
    "id": 4,
    "instruction": """以下の質問文（input）に対して回答（output）してください。
質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。""",
    "input": """営業活動のレポート書き起こしを選択してください。次のうち、最も適切なものを選択してください。
A) 今回のプロジェクトでは、会議室を予約することができました。
B) 今回のプロジェクトでは、注文を確定していただきました。
C) 今回のプロジェクトでは、協力いただけるか確認していただきました。
D) 今回のプロジェクトでは、レポートを請求していただきました。""",
    "output": "B"
  }
]

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE=1 #temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS=10000
LOOP_NUM = 10 #1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。"},
         {"role": "user", "content": PROMPT_TEMPLATE}],
        temperature=TEMPERATURE,#temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,#0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,#-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3 #-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
        )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            #output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

 [{"id": 1, "instruction": "以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。", "input": "会社が宣伝品を販売しようと考えています。この場合、一般消費者が受け入れやすい商品の KIND は次のうちどれですか？\nA) 食品\nB) ホテル\nC) ブランド記念品\nD) 教育", "output": "C"}, {"id": 2, "instruction": "以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。", "input": "以下の4つの選択肢のうち、ビジネスで最も重要な것はどれですか？\nA) 時間管理\nB) 情報発信\nC) マーケティング\nD) プレゼンテーション", "output": "A"}, {"id": 3, "instruction": "以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。", "input": "次の選択肢のうち、キャッシュフローを改善する方法はどれですか？\nA) 支払いを早める\nB) 売上高を増やす\nC) 利益を最大化する\nD) 会社の名前を変更する", "output": "B"}, {"id": 4, "instruction": "以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C,Dの4つの選択肢が含まれています。回答はA,B,C,Dのいずれかで回答してください。", "input": "次のうち、企業がお客様の声を顧みたい場合に最も適切な方法はどれですか？\nA) パンフレットを配布する\nB) 顧客を勧誘する\nC) アンケート調査を行う\nD) 宣伝を続ける", "output": "C"}, {"id": 5, "instruction": "以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,

,id,instruction,input,output
0,1,"以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C...",会社が宣伝品を販売しようと考えています。この場合、一般消費者が受け入れやすい商品の KIND...,C
1,2,"以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C...",以下の4つの選択肢のうち、ビジネスで最も重要な것はどれですか？\nA) 時間管理\nB) 情...,A
2,3,"以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C...",次の選択肢のうち、キャッシュフローを改善する方法はどれですか？\nA) 支払いを早める\nB...,B
3,4,"以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C...",次のうち、企業がお客様の声を顧みたい場合に最も適切な方法はどれですか？\nA) パンフレット...,C
4,5,"以下の質問文（input）に対して回答（output）してください。\n質問文にはA,B,C...",次のうち、会社の新製品開発におけるリスクを軽減するために役立つものはどれですか？\nA) 商...,D


##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")

File saved as: /generated_data_2024-05-15_07-58-51.xlsx
